# Tweepy Handles Scraper

## INIT

In [1]:
import tweepy
import pandas as pd
import numpy as np

In [2]:
# assign the values accordingly 
consumer_key = "4UK25l7CHvfv47CmvWtrSGBYW" 
consumer_secret = "WUO6GHT5NW506qqa3lSt1F836kWHXUf2sM5EjwFcWU1nStwGVT" 
access_token = "3097042092-7Kxie8uQDEYfS0BGNYoc93i2oRHrSW4toeibhU4" 
access_token_secret = "rCBPLXlTEzjX4J6X0pSrOHAcs85iP9zJiEXGXrCDwYajo"

In [3]:
# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# set access to user's access key and access secret  
auth.set_access_token(access_token, access_token_secret) 
  
# calling the api  
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
df=pd.read_excel(r'../input/SnP_500_CEO_Names.xlsx')
df.dropna(inplace=True)
df.drop(['CEO Name', 'YEAR'], axis=1, inplace=True)
df.drop_duplicates(subset=['Company'], inplace=True)
df.shape
#df['ceos']= df[['CEO Name', 'Company']].apply(lambda x: ' '.join(x), axis=1)
#df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.shape

(782, 3)

In [5]:
df.columns

Index(['Company', 'GVKEY', 'EXECID'], dtype='object')

In [6]:
df.head()

,Company,GVKEY,EXECID
0,ADC TELECOMMUNICATIONS INC,1013,10203
4,AMERICAN AIRLINES GROUP INC,1045,14591
17,PINNACLE WEST CAPITAL CORP,1075,10739
30,ABBOTT LABORATORIES,1078,14300
43,ADVANCED MICRO DEVICES,1161,20918


In [7]:
from collections import Counter
cnt = Counter()
for text in df["Company"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(30)

[('INC', 384),
 ('CORP', 193),
 ('CO', 78),
 ('&', 35),
 ('GROUP', 32),
 ('ENERGY', 30),
 ('PLC', 30),
 ('HOLDINGS', 23),
 ('FINANCIAL', 21),
 ('LTD', 15),
 ('TECHNOLOGIES', 14),
 ('INTERNATIONAL', 13),
 ('INTL', 13),
 ('COS', 10),
 ('BRANDS', 9),
 ('-CL', 9),
 ('RESOURCES', 9),
 ('AMERICAN', 8),
 ('HEALTH', 8),
 ('TECHNOLOGY', 8),
 ('A', 8),
 ('SYSTEMS', 8),
 ('INDUSTRIES', 7),
 ('COMMUNICATIONS', 7),
 ('NATIONAL', 7),
 ('REALTY', 7),
 ('GLOBAL', 6),
 ('SERVICES', 6),
 ('SOLUTIONS', 5),
 ('GRP', 5)]

In [8]:
df['CompanyName']= df['Company']

In [9]:
df['Company']= df['Company'].str.replace(r"\(.*\)","")
df['Company']= df['Company'].str.replace(' -CL A', '')
df['Company']= df['Company'].str.replace(' -CL B', '')
df['Company']= df['Company'].str.replace('/DE', '')
df['Company']= df['Company'].str.replace('/ME', '')
#df['Company']= df['Company'].str.replace(r'\([^()]*\)', '')
df['Company']= df['Company'].str.replace('ARCHER-DANIELS-MIDLAND CO', 'ARCHER-DANIELS-MIDLAND')
df['Company']= df['Company'].str.strip()

In [10]:
cp=[]
for i in df.Company:
    a=i.split()
    if len(a)==6:
        i=i.replace(' CORP', '')
        cp.append(i)
        #print(i) 
    elif len(a)>=3:
        #i=i.join(content.split(' ')[:-1])
        i= i.replace(' INC', '')
        i= i.replace(' INTL', '')
        i= i.replace(' CORP', '')
        i= i.replace(' CO', '')
        i= i.replace(' COS', '')
        i= i.replace(' PLC', '')
        i= i.replace(' GRP', '')
        i= i.replace(' GROUP', '')
        
        i= i.replace(' SVCS', '')
        i= i.replace(' PLC', '')
        i= i.replace('\/.*$', '')
        cp.append(i)
    else:
        cp.append(i)
               
    
len(cp)

782

In [12]:
df['Company']=cp

In [13]:
df['Company'] = df['Company'].str.replace('-', ' ')
df['Company'] = df['Company'].str.replace('VERIZONMMUNICATIONS', 'VERIZON COMMUNICATIONS')
df['Company']= df.Company.str.strip()

In [16]:
tdf=pd.DataFrame()
companies=[]
gvk=[]
handles=[]
descriptions=[]
verified=[]

badnames=[]
badgv=[]

for i in range(len(df)):
    users = api.search_users(df.iloc[i]['Company'])                     
    lst=[]
    for user in users:
        lst.append(user.screen_name)
    if len(lst)==0:
        badnames.append(df.iloc[i]['CompanyName'])
        badgv.append(df.iloc[i]['GVKEY'])
        print('--------------------------------- ',df.iloc[i]['Company'],'---------------------------------' )
    for j in lst:
        obj=api.get_user(j)
        result=obj.verified
        if result==True:
            companies.append(df.iloc[i]['CompanyName'])
            gvk.append(df.iloc[i]['GVKEY'])
            handles.append(j)
            descriptions.append(obj.description)
            verified.append(result)
            print(df.iloc[i]['Company'],'----', j)

---------------------------------  ADC TELECOMMUNICATIONS ---------------------------------
AMERICAN AIRLINES ---- AmericanAir
AMERICAN AIRLINES ---- AAarena
AMERICAN AIRLINES ---- uptin
AMERICAN AIRLINES ---- rosebaca
AMERICAN AIRLINES ---- LATAMAirlines
---------------------------------  PINNACLE WEST CAPITAL ---------------------------------
ABBOTT LABORATORIES ---- AbbottNews
ADVANCED MICRO DEVICES ---- AMD
ADVANCED MICRO DEVICES ---- AMDGaming
ADVANCED MICRO DEVICES ---- AMD_UK
AETNA INC ---- Aetna
AIR PRODUCTS & CHEMICALS ---- airproducts
ALASKA AIR ---- AlaskaAir
ALASKA AIR ---- RavnAlaska
HONEYWELL INTERNATIONAL ---- honeywell
---------------------------------  ALLTEL CORP ---------------------------------
ALCOA INC ---- Alcoa
AMERICAN ELECTRIC POWER ---- AEPnews
AMERICAN EXPRESS ---- AmericanExpress
AMERICAN EXPRESS ---- AmexUK
AMERICAN EXPRESS ---- AmexBusiness
AMERICAN EXPRESS ---- AskAmex
AMERICAN EXPRESS ---- AmexBusinessCan
AMERICAN EXPRESS ---- theamexgolf
AMERICAN EXPRE

EL PASO ---- EPCPublicHealth
EL PASO ---- 600espnelpaso
EMERSON ELECTRIC ---- Emerson_News
EMERSON ELECTRIC ---- EmersonComRes
WEATHERFORD ---- WeatherfordCorp
WEATHERFORD ---- willweatherford
WEATHERFORD ---- sincerelyash
WEATHERFORD ---- Weatherford5
ATMOS ENERGY ---- atmosenergy
EQUIFAX INC ---- Equifax
---------------------------------  EXPEDITORS WASH ---------------------------------
EXXON MOBIL ---- exxonmobil
EXXON MOBIL ---- ExxonMobil_EU
EXXON MOBIL ---- exxonmobil_aus
EXXON MOBIL ---- ExxonMobil_UK
EXXON MOBIL ---- exxonmobil_sg
EXXON MOBIL ---- ExxonMobil_PNG
EXXON MOBIL ---- exxonmobilbmt
EXXON MOBIL ---- XOM_Chemical
EXXON MOBIL ---- exxonmobil_ind
EXXON MOBIL ---- XOM_NatGas
EXXON MOBIL ---- XOMFoundation
EXXON MOBIL ---- exxonmobil_th
EXXON MOBIL ---- ExxonMobil_ID
EXXON MOBIL ---- ExxonMobil_GER
NEXTERA ENERGY ---- NextEraEnergyR
NEXTERA ENERGY ---- nexteraenergy
NEXTERA ENERGY ---- pmusser
FAMILY DOLLAR STORES ---- myfamilydollar
---------------------------------  FED

S&P GLOBAL ---- SPGlobal
S&P GLOBAL ---- SPGlobalRatings
S&P GLOBAL ---- SPGMarketIntel
S&P GLOBAL ---- Leonardo_live
S&P GLOBAL ---- Leonardo_UK
S&P GLOBAL ---- pmnewsnigeria
S&P GLOBAL ---- CHEXNewswatch
S&P GLOBAL ---- taykuy
S&P GLOBAL ---- JDBlum23
---------------------------------  MEDCO HEALTH SOLUTIONS ---------------------------------
MEDTRONIC PLC ---- Medtronic
CVS HEALTH ---- CVSHealth
CVS HEALTH ---- CVSHealthJobs
CVS HEALTH ---- CVSClassicGolf
CVS HEALTH ---- CVSHealthIR
CVS HEALTH ---- CVSCareers
CVS HEALTH ---- cvspharmacy
CVS HEALTH ---- CVSinAction
CVS HEALTH ---- ndegreve
CVS HEALTH ---- AlanLotvinMD
MERCK & ---- Merck
MERCK & ---- merckgroup
MERCK & ---- MerckforMothers
MERCK & ---- USMerckProducts
MERCK & ---- MMercuriadis
MERCK & ---- MerckManualPro
MERCK & ---- merck50
MERCK & ---- MerckManualHome
MEREDITH CORP ---- MeredithCorp
MERRILL LYNCH & ---- MerrillLynch
MERRILL LYNCH & ---- SallieKrawcheck
MERRILL LYNCH & ---- MarshallM13
MICRON TECHNOLOGY ---- MicronTec

GLOBE LIFE ---- TIME
GLOBE LIFE ---- TEXPAMAN
TRIBUNE MEDIA ---- tribunemedia
TRIBUNE MEDIA ---- HeraldTribune
TRIBUNE MEDIA ---- YassTribune
TRIBUNE MEDIA ---- LauraPavinNews
TRIBUNE MEDIA ---- TamayoMedia
TRIBUNE MEDIA ---- PaulSkrbina
TYCO INTERNATIONAL ---- johnsoncontrols
TYSON FOODS ---- TysonFoods
TYSON FOODS ---- TysonFoodsCEO
TYSON FOODS ---- TysonBrand
TYSON FOODS ---- kelly_roliard
---------------------------------  UNITED AIRLINES HOLDINGS ---------------------------------
AMEREN CORP ---- AmerenCorp
UNION PACIFIC ---- UnionPacific
UNION PACIFIC ---- APALAnational
UNION PACIFIC ---- sujiseam
UNITEDHEALTH ---- UnitedHealthGrp
UNITED PARCEL SERVICE ---- UPS
RAYTHEON TECHNOLOGIES ---- RaytheonTech
RAYTHEON TECHNOLOGIES ---- RaytheonPolicy
RAYTHEON TECHNOLOGIES ---- Raytheon_Jobs
RAYTHEON TECHNOLOGIES ---- TheHall
RAYTHEON TECHNOLOGIES ---- UTC
UNIVERSAL HEALTH ---- CHGlobalHealth
UNIVERSAL HEALTH ---- WHOMalaysia
---------------------------------  VORNADO REALTY TRUST --------

NEWS CORP ---- NewsUK
NEWS CORP ---- superracing
NEWS CORP ---- CharlesSchwab
NEWS CORP ---- manilabulletin
NEWS CORP ---- BurberryCorp
NEWS CORP ---- billbirtles
NEWS CORP ---- CanonUSA
NEWS CORP ---- DynCorpIntl
NEWS CORP ---- BofA_News
---------------------------------  MALLINCKRODT PLC ---------------------------------
PTC INC ---- PTC
TWITTER INC ---- Twitter
TWITTER INC ---- TwitterData
TWITTER INC ---- WWE
TWITTER INC ---- SEGA
TWITTER INC ---- INCIndia
TWITTER INC ---- amazon
TWITTER INC ---- acnnews
TWITTER INC ---- GoPro
TWITTER INC ---- WWEUniverse
TWITTER INC ---- MarvelMusicInc
TWITTER INC ---- MarvelUK
TWITTER INC ---- amazonIN
TWITTER INC ---- CanonUSA
TWITTER INC ---- blockbuster
TWITTER INC ---- ASUS
TWITTER INC ---- Stussy
TWITTER INC ---- InterFM897
TWITTER INC ---- HondaCanada
TWITTER INC ---- ANSYS
CAESARS ENTERTAINMENT ---- CaesarsEnt
CAESARS ENTERTAINMENT ---- Caesars_Rewards
CABOT OIL & GAS ---- CabotOG
---------------------------------  DENBURY RESOURCES ------

---------------------------------  EVEREST RE LTD ---------------------------------
---------------------------------  DIAMOND OFFSHRE DRILLING ---------------------------------
DAVITA INC ---- DaVita
---------------------------------  QWESTMMUNICATION ---------------------------------
HENRY SCHEIN ---- HenrySchein
---------------------------------  SANDISK CORP ---------------------------------
WATERS CORP ---- WatersCorp
NETAPP INC ---- NetApp
NETAPP INC ---- NetAppME
NETAPP INC ---- NetAppEMEA
NETAPP INC ---- NetAppANZ
NETAPP INC ---- NetAppAPAC
NETAPP INC ---- NetAppSchweiz
NETAPP INC ---- NetAppAustria
NETAPP INC ---- NetAppFrance
CITRIX SYSTEMS ---- citrix
CITRIX SYSTEMS ---- tminahan
CITRIX SYSTEMS ---- DavidJHenshall
HARTFORD FINANCIAL SERVICES ---- TheHartford
ALEXION PHARMACEUTICALS ---- AlexionPharma
ALTABA INC ---- Yahoo
ALTABA INC ---- YahooSG
DEAN FOODS ---- TysonFoodsCEO
ANSYS INC ---- ANSYS
E TRADE FINANCIAL ---- etrade
ABERCROMBIE & FITCH ---- Abercrombie
ABERCROMBIE &

NYSE EURONEXT ---- nokia
FACEBOOK INC ---- fbnewsroom
FACEBOOK INC ---- Facebook
FACEBOOK INC ---- facebookapp
FACEBOOK INC ---- RazerSupport
FACEBOOK INC ---- Alden_Gonzalez
FACEBOOK INC ---- SimonPearceInc
FACEBOOK INC ---- hemeon
FACEBOOK INC ---- wa_status
---------------------------------  DIAMONDBACK ENERGY ---------------------------------
PHILLIPS 66 ---- Phillips66Co
PHILLIPS 66 ---- Phillips66Gas
---------------------------------  EMBARQ CORP ---------------------------------
WINDSTREAM HOLDINGS ---- Windstream
WYNDHAM DESTINATIONS ---- wyndestinations
WESTERN UNION ---- WesternUnion
WESTERN UNION ---- WesternUnionCEO
WESTERN UNION ---- wucanada
WESTERN UNION ---- UnionPacific
WESTERN UNION ---- WGAWest
WESTERN UNION ---- WUBusiness
WESTERN UNION ---- MikeValdesFauli
WESTERN UNION ---- WUStopFraud
TIME WARNER CABLE ---- SpectrumSN
TIME WARNER CABLE ---- SportsNetLA
TIME WARNER CABLE ---- Mike_Bresnahan
TIME WARNER CABLE ---- spectrumcenter
---------------------------------  B

In [17]:
columns = ['GVKEY', 'Company', 'Handle', 'Twitter_Descr', 'verified']
tdf = pd.DataFrame(list(zip(gvk, companies, handles, descriptions, verified)), columns =columns) 
tdf.dropna(inplace=True)
tdf= tdf[tdf['verified']!=False]
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1304 entries, 0 to 1303
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GVKEY          1304 non-null   int64 
 1   Company        1304 non-null   object
 2   Handle         1304 non-null   object
 3   Twitter_Descr  1304 non-null   object
 4   verified       1304 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 52.2+ KB


In [28]:
tdf['Company'].value_counts()[:60]

JOHNSON & JOHNSON               19
JONES GROUP INC                 19
SMITH (A.O.)                    19
TWITTER INC                     19
NEW YORK TIMES CO  -CL A        19
TIFFANY & CO                    18
COCA-COLA CO                    18
DIRECTV                         18
NETFLIX INC                     17
AT&T INC                        16
BANK OF AMERICA CORP            16
NEWS CORP                       16
LILLY (ELI) & CO                15
EXXON MOBIL CORP                14
MARRIOTT INTL INC               14
CBRE GROUP INC                  14
EBAY INC                        14
EATON CORP PLC                  13
MCCORMICK & CO INC              13
DISNEY (WALT) CO                13
CISCO SYSTEMS INC               13
DELL TECHNOLOGIES INC           13
AMERICAN EXPRESS CO             12
GENERAL MOTORS CO               11
LINCOLN NATIONAL CORP           11
NOBLE CORP PLC                  11
GENERAL ELECTRIC CO             11
FORD MOTOR CO                   11
LOCKHEED MARTIN CORP

In [144]:
dfs=tdf[tdf['Company'].map(tdf['Company'].value_counts()) == 1]
len(dfs.Company.unique())

178

In [ ]:
# w/o string corrections
# 666 handles
# 299 badnames
# 264 verified handles

In [83]:
list_dfs=[bdf.iloc[:19], bdf.iloc[19:38], bdf.iloc[38:57], bdf.iloc[57:76], bdf.iloc[76:95], bdf.iloc[95:]]
xls_path= r"D:\OneDrive - Indian School of Business\Data\badnames.xlsx"

def save_xls(list_dfs, xls_path):
    with pd.ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

save_xls(list_dfs, xls_path)

In [15]:
tdf.to_csv("../output/Company_handles_tweepy_lookup_WRDSInput.csv", index=False)

In [101]:
dfs.to_csv(r"D:\OneDrive - Indian School of Business\Projects\Mindset\tweets\TWITTERHANDLESSEARCH\output\verified\Company\tweepy_verified_single_company_handles.csv", index=False)

In [98]:
dfs1= pd.read_csv(r"D:\OneDrive - Indian School of Business\Projects\Mindset\tweets\TWITTERHANDLESSEARCH\output\verified\Company\tweepy_verified_single_company_handles_2.csv")
dfs= pd.concat([dfs, dfs1], axis=0)
dfs.shape

(224, 5)

In [99]:
dfs.drop_duplicates(inplace=True)
dfs.shape

(216, 5)

In [100]:
dfs.drop_duplicates(subset='GVKEY', inplace=True)
dfs.shape

(178, 5)

In [102]:
tdf

,GVKEY,Company,Handle,Twitter_Descr,verified
0,1045,AMERICAN AIRLINES GROUP INC,AmericanAir,Official Twitter for the world’s largest airli...,True
1,1045,AMERICAN AIRLINES GROUP INC,AAarena,AmericanAirlines Arena in Miami is the top Sou...,True
2,1045,AMERICAN AIRLINES GROUP INC,uptin,"I tell stories about tech, money & cultures. A...",True
3,1045,AMERICAN AIRLINES GROUP INC,rosebaca,Digital Media @ American Airlines. Former phot...,True
4,1045,AMERICAN AIRLINES GROUP INC,LATAMAirlines,Bienvenido a la cuenta oficial de LATAM Airlin...,True
...,...,...,...,...,...
1299,260774,CBRE GROUP INC,CBREResearch,We deliver industry-leading insights that enab...,True
1300,260774,CBRE GROUP INC,CBREDFW,The leading commercial real estate and investm...,True
1301,260774,CBRE GROUP INC,cbreAustralia,A globally integrated commercial real estate a...,True
1302,260774,CBRE GROUP INC,cbrechicago,The leading commercial real estate and investm...,True


In [114]:
tdf['value_counts']= tdf.groupby(['GVKEY'])['GVKEY'].transform('count')
tdf

,GVKEY,Company,Handle,Twitter_Descr,verified,value_counts
0,1045,AMERICAN AIRLINES GROUP INC,AmericanAir,Official Twitter for the world’s largest airli...,True,5
1,1045,AMERICAN AIRLINES GROUP INC,AAarena,AmericanAirlines Arena in Miami is the top Sou...,True,5
2,1045,AMERICAN AIRLINES GROUP INC,uptin,"I tell stories about tech, money & cultures. A...",True,5
3,1045,AMERICAN AIRLINES GROUP INC,rosebaca,Digital Media @ American Airlines. Former phot...,True,5
4,1045,AMERICAN AIRLINES GROUP INC,LATAMAirlines,Bienvenido a la cuenta oficial de LATAM Airlin...,True,5
...,...,...,...,...,...,...
1299,260774,CBRE GROUP INC,CBREResearch,We deliver industry-leading insights that enab...,True,14
1300,260774,CBRE GROUP INC,CBREDFW,The leading commercial real estate and investm...,True,14
1301,260774,CBRE GROUP INC,cbreAustralia,A globally integrated commercial real estate a...,True,14
1302,260774,CBRE GROUP INC,cbrechicago,The leading commercial real estate and investm...,True,14


In [116]:
tdf['value_counts'].unique()

array([ 5,  1,  3,  2, 12, 10,  6,  4,  9, 18, 13, 14,  8, 11, 19, 15, 16,
        7, 17], dtype=int64)

In [153]:
l1=[3,5]
l2=[4,6,7]
l3=[8,9,10, 11, 12,13,14]

In [154]:
df1=tdf[tdf['value_counts']==2]
len(df1.Company.unique())

48

In [155]:
df2=tdf[tdf['value_counts'].isin(l1)]
len(df2.Company.unique())

44

In [156]:
df3=tdf[tdf['value_counts'].isin(l2)]
len(df3.Company.unique())

45

In [157]:
df4=tdf[tdf['value_counts'].isin(l3)]
len(df4.Company.unique())

43

In [158]:
df5=tdf[tdf['value_counts']>14]
len(df5.Company.unique())

13

In [159]:
df1.shape

(96, 6)

In [160]:
df2.shape

(152, 6)

In [161]:
df3.shape

(214, 6)

In [162]:
df4.shape

(435, 6)

In [163]:
df5.shape

(229, 6)

In [168]:
list_dfs=[df5,df4,df1,df2,df3]
xls_path= r"D:\OneDrive - Indian School of Business\Data\multiplehandles.xlsx"

def save_xls(list_dfs, xls_path):
    with pd.ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

save_xls(list_dfs, xls_path)

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
